In [ ]:
from mnist import MNIST
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from threading import Thread
from operator import itemgetter
from sklearn.ensemble import RandomForestClassifier
import torch
import torchvision                                 
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn                              
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

In [ ]:
class Perceptron(nn.Module):
    def __init__(self, d, K):
        super(Perceptron, self).__init__()
        self.model = nn.Linear(d, K)
    
    def forward(self,x):
        batch_size = x.shape[0]
        x = x.view(batch_size, -1)
        x = self.model(x.float())
        return x

In [ ]:
class CustomGeneticForestClassifier:
    def __init__(self, N, generation_number, class_percentage):
        self.N = N
        self.generation_number = generation_number
        self.class_percentage = class_percentage
        self.trained_trees = []
        self.slp = None
        self.prediction_tree = None
        self.one_hot_encoder = None
        self.SEED = 0          
        self.EPOCHS = 20          
        self.LR = 0.01            # learning rate
        self.MOMENTUM = 0.9       # momentum for the optimizer
        self.WEIGHT_DECAY = 1e-5  #
        self.GAMMA = 0.1          # learning rate schedular
        self.BATCH_SIZE = 32      # number of images to load per iteration
        
        self.train_tree_batch = 20
        self.mutation_rate = 0.2
        self.population_list = None

    def train_net(self):
        self.slp.train()
        epoch_loss = 0.0
        for xt, rt in self.train_loader:
            xt, rt = xt.to(self.device), rt.to(self.device)
            self.optimizer.zero_grad() 
            yt = self.slp(xt)
            loss = self.loss_fn(yt, rt)
            loss.backward()
            self.optimizer.step()
            epoch_loss += loss.item()
        return epoch_loss
        
    def train_tree(self, filter_label, state_counter, tree_list):
        
        y_train_subtree = self.y_train[self.y_train.label.isin(filter_label)]
        X_train_subtree = self.X_train.loc[y_train_subtree.index.values.tolist()]
        
        dtc = DecisionTreeClassifier(random_state=state_counter)
        dtc = dtc.fit(X_train_subtree,y_train_subtree)
        y_valid_filtered= self.y_valid[self.y_valid.label.isin(filter_label)]
        X_valid_filtered = self.X_valid.loc[y_valid_filtered.index]
        y_pred = dtc.predict(X_valid_filtered)
        tree_accuracy = metrics.accuracy_score(y_valid_filtered, y_pred)
        print(state_counter+1, end=" ")
        tree_list.append({"tree": dtc, "accuracy": tree_accuracy, "filter_label": filter_label})   
        
    def fit_trees(self, filter_labels, tree_list):
        print("Started to train {} trees.".format(len(filter_labels)))
        print("Trained Trees: ", end="")
        state_counter = 0
        train_threads = []
        for filter_label in filter_labels:
            train_threads.append(Thread(target=self.train_tree, args=[filter_label, state_counter, tree_list]))
            state_counter += 1
        for thread_index in range(0, len(train_threads), self.train_tree_batch ):
            current_train_threads = train_threads[thread_index:thread_index+self.train_tree_batch ]
            for train_thread in current_train_threads:
                train_thread.start()
            for train_thread in current_train_threads:
                train_thread.join()
        print("\n{} Trees are trained.".format(len(filter_labels)))
                
    def train_slp(self, one_hot_encoded_predictions):
        print("Started to train SLP.")
        self.d = one_hot_encoded_predictions.shape[1]      # number of input features 
        
        # manual seed to reproduce same resultsnet
        torch.manual_seed(self.SEED)
        
        self.slp = Perceptron(self.d,self.K)
        cuda = torch.cuda.is_available()  
        self.device = torch.device("cuda:0" if cuda else "cpu")
        self.slp.to(self.device)
        self.loss_fn = nn.CrossEntropyLoss()
        self.optimizer = optim.SGD(self.slp.parameters(), lr=self.LR, momentum=self.MOMENTUM, weight_decay=self.WEIGHT_DECAY)
        self.scheduler = lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=self.GAMMA) #CHECK THIS
        
        train_target = torch.tensor(self.y_train.values.flatten().astype(np.int32)).long()

        train = torch.tensor(one_hot_encoded_predictions) 

        train_tensor = torch.utils.data.TensorDataset(train, train_target) 
        self.train_loader = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = self.BATCH_SIZE, shuffle = True, num_workers=8)
        
        # train the network
        for epoch in range(1,self.EPOCHS+1):
            self.train_net()
        print("SLP is trained.")
        
    def one_hot_encode(self, total_predictions):
        self.one_hot_encoder = OneHotEncoder(handle_unknown='ignore') 
        self.one_hot_encoder.fit(total_predictions)
        
        one_hot_encoded_predictions = self.one_hot_encoder.transform(total_predictions).toarray() 
        return one_hot_encoded_predictions

    
    def train_prediction_tree(self, one_hot_encoded_predictions):
        self.prediction_tree = DecisionTreeClassifier(random_state=200)
        self.prediction_tree = self.prediction_tree.fit(one_hot_encoded_predictions, self.y_train)
        
        
    def fit(self, X_train, y_train):
        self.X_train, self.X_valid, self.y_train, self.y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0)  # Train-test split pairs
        self.label_count = len(y_train.label.unique())
        self.sample_count = y_train.shape[0]
        self.K = self.label_count              # number of output features
        
        self.population_list = self.genetic_find_parameters()
        last_population = self.population_list[-1]
        
        self.trained_trees = [member['tree'] for member in last_population]
        
        total_predictions = self.forest_trees_predict(self.X_train)
        one_hot_encoded_predictions = self.one_hot_encode(total_predictions)
        self.train_slp(one_hot_encoded_predictions)
        self.train_prediction_tree(total_predictions)
        
    def model_analysis(self, X_train, y_train, X_test, y_test):
        self.X_train, self.X_valid, self.y_train, self.y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0)  # Train-test split pairs
        self.label_count = len(y_train.label.unique())
        self.sample_count = y_train.shape[0]
        self.K = self.label_count              # number of output features
        
        self.population_list = self.genetic_find_parameters()
        
        majority_accuracies = []
        slp_accuracies = []
        prediction_tree_accuracies = []
        population_mean_accuracies = []
        
        for population in self.population_list:
            
            population_mean_accuracy = np.mean(np.asarray([member['accuracy'] for member in population]))
            population_mean_accuracies.append(population_mean_accuracy)

            self.trained_trees = [member['tree'] for member in population]
            total_predictions = self.forest_trees_predict(self.X_train)
            one_hot_encoded_predictions = self.one_hot_encode(total_predictions)
            self.train_slp(one_hot_encoded_predictions)
            self.train_prediction_tree(total_predictions)
            
            majority_voting_pred = self.majority_voting_predict(X_test)
            slp_pred = self.slp_predict(X_test)
            prediction_tree_predict = self.prediction_tree_predict(X_test)
            
            majority_accuracy = metrics.accuracy_score(y_test, majority_voting_pred)
            slp_accuracy = metrics.accuracy_score(y_test, slp_pred)
            prediction_tree_accuracy = metrics.accuracy_score(y_test, prediction_tree_predict)
            
            majority_accuracies.append(majority_accuracy)
            slp_accuracies.append(slp_accuracy)
            prediction_tree_accuracies.append(prediction_tree_accuracy)
        generation_numbers = np.arange(len(self.population_list))
        return generation_numbers, prediction_tree_accuracies, slp_accuracies, majority_accuracies, population_mean_accuracies
        
    
    def majority_voting_predict(self, X_test):
        total_predictions = self.forest_trees_predict(X_test)
        # Majority Voting
        predicted_values = []
        for row in total_predictions:
            majority_vote = np.bincount(row).argmax()
            predicted_values.append(majority_vote)
        y_pred_class = np.asarray(predicted_values)
        return y_pred_class
    
    def slp_predict(self, X_test):
        total_predictions = self.forest_trees_predict(X_test)
        # SLP
        one_hot_encoded_predictions = self.one_hot_encoder.transform(total_predictions).toarray() 
        test = torch.tensor(one_hot_encoded_predictions) 
        y_pred = self.slp(test.to(self.device))
        y_pred = y_pred.cpu().detach().numpy()
        y_pred_class = np.asarray([np.argmax(pred) for pred in y_pred])
        return y_pred_class
    
    def prediction_tree_predict(self, X_test):
        total_predictions = self.forest_trees_predict(X_test)
        # Prediction Tree
        y_pred_class = self.prediction_tree.predict(total_predictions)
        return y_pred_class
    
    def forest_trees_predict(self, X_test):
        total_predictions = self.trained_trees[0].predict(X_test)
        for i in range(1, self.N):
            total_predictions = np.vstack([total_predictions, self.trained_trees[i].predict(X_test)])
        total_predictions = np.transpose(total_predictions)
        return total_predictions
    
    
    # Genetic algorithm  
    def generate_parent_samples(self):
        generation = []
        for i in range(self.N):
            generation.append(np.random.choice(range(self.label_count), round(self.label_count*self.class_percentage), replace=False))
        return generation  


    def genetic_find_parameters(self):
        print("Genetic algorithm is started.")
        generation = self.generate_parent_samples()
        population_list = []
        for i in range(self.generation_number+1):
            print("Generation:",i)
            trained_tree_results = []
            self.fit_trees(generation, trained_tree_results)
            population_list.append(trained_tree_results)
            generation = self.evolve(trained_tree_results)
        print("\nGenetic algorithm is ended.")
        return population_list
    
        
    def evolve(self, trained_tree_results):
        
        trained_tree_results_sorted = sorted(trained_tree_results, key=itemgetter("accuracy"), reverse=True)
        
        next_generation = []
        
        # Elitism
        next_generation.append(trained_tree_results_sorted[0]["filter_label"])
        for i in range(1, len(trained_tree_results)):
            parent_1 = self.tournament(trained_tree_results)
            parent_2 = self.tournament(trained_tree_results)
            child = self.crossover(parent_1, parent_2)
            child = self.mutate(child)  
            next_generation.append(child)        
        return next_generation

    
    def crossover(self, parent1, parent2):
        parents_merged = np.unique(np.append(parent1, parent2))
        child = np.random.choice(parents_merged, len(parent1), replace=False)
        return np.sort(child)

    
    def mutate(self, child):
        mutated_child = []
        if len(child) == self.label_count:
            return child
        non_existing_labels = []
        for label in range(self.label_count):
            if label not in child:
                non_existing_labels.append(label)
        for gen in child:
            if np.random.random() < self.mutation_rate:
                selected_label_index = np.random.randint(len(non_existing_labels))
                mutated_child.append(non_existing_labels.pop(selected_label_index))
            else:
                mutated_child.append(gen)
        return np.sort(mutated_child)

    def tournament(self, generation):
        accuracies = np.asarray([tree["accuracy"] for tree in generation])
        accuracies -= np.min(accuracies)
        probabilities = np.asarray(accuracies)/sum(accuracies)
        selected = np.random.choice(generation, 1, p=probabilities)[0]["filter_label"]
        return selected

In [ ]:
np.random.seed(60) # reproducability

def mnist_dataset_read(path):
    mndata = MNIST(path)

    # read training images and corresponding labels
    tr_images, tr_labels = mndata.load_training()
    # read test images and corresponding labels
    tt_images, tt_labels = mndata.load_testing()

    # convert lists into numpy format and apply normalization
    tr_images = np.array(tr_images) / 255. # shape (60000, 784)
    tr_labels = np.array(tr_labels)         # shape (60000,)
    tt_images = np.array(tt_images) / 255. # shape (10000, 784)
    tt_labels = np.array(tt_labels)         # shape (10000,)

    columns_images = ['p{}'.format(i+1) for i in range(784)]
    X_train = pd.DataFrame(data=tr_images, columns=columns_images)
    y_train = pd.DataFrame(data=tr_labels, columns=['label'])
    X_test = pd.DataFrame(data=tt_images, columns=columns_images)
    y_test = pd.DataFrame(data=tt_labels, columns=['label'])
    return X_train, X_test, y_train, y_test

In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def load_data(btch):
    labels = btch[b'labels']
    imgs = btch[b'data'].reshape((-1, 32, 32, 3))
    
    res = []
    for ii in range(imgs.shape[0]):
        img = imgs[ii].copy()
        img = np.fliplr(np.rot90(np.transpose(img.flatten().reshape(3,32,32)), k=-1))
        res.append(img)
    imgs = np.stack(res)
    return labels, imgs

def load_data_cifar():
    batch1 = unpickle("Datasets/cifar-10-batches-py/data_batch_1")
    batch2 = unpickle("Datasets/cifar-10-batches-py/data_batch_2")
    batch3 = unpickle("Datasets/cifar-10-batches-py/data_batch_3")
    batch4 = unpickle("Datasets/cifar-10-batches-py/data_batch_4")
    batch5 = unpickle("Datasets/cifar-10-batches-py/data_batch_5")
    test_batch = unpickle("Datasets/cifar-10-batches-py/test_batch")
    
    pixel_num = 32*32*3
    x_train_l = []
    y_train_l = []
    for ibatch in [batch1, batch2, batch3, batch4, batch5]:
        labels, imgs = load_data(ibatch)
        x_train_l.append(imgs)
        y_train_l.extend(labels)
    x_train = np.vstack(x_train_l)
    y_train = np.vstack(y_train_l)
    
    x_test_l = []
    y_test_l = []
    labels, imgs = load_data(test_batch)
    x_test_l.append(imgs)
    y_test_l.extend(labels)
    x_test = np.vstack(x_test_l)
    y_test = np.vstack(y_test_l)
    
    del batch1, batch2, batch3, batch4, batch5, test_batch
    
    x_train, x_test = x_train.reshape(-1, pixel_num), x_test.reshape(-1, pixel_num)
    
    columns_images = ['p{}'.format(i+1) for i in range(pixel_num)]
    X_train = pd.DataFrame(data=x_train, columns=columns_images)
    y_train = pd.DataFrame(data=y_train, columns=['label'])
    X_test = pd.DataFrame(data=x_test, columns=columns_images)
    y_test = pd.DataFrame(data=y_test, columns=['label'])
    
    return X_train, y_train, X_test, y_test

In [ ]:
def plot_genetic_analysis(generation_numbers, prediction_tree_accuracies, slp_accuracies, majority_accuracies, population_mean_accuracies, dataset_name):
    plt.figure(figsize=(20, 10))
    plt.plot(generation_numbers, prediction_tree_accuracies, marker="o", label = "Prediction Tree")
    plt.plot(generation_numbers, slp_accuracies, marker="o", label = "SLP")
    plt.plot(generation_numbers, majority_accuracies, marker="o", label = "Majority Voting")
    plt.plot(generation_numbers, population_mean_accuracies, marker="o", label = "Population Mean*")
    plt.title("Accuracies of Generations ({} Dataset)\n".format(dataset_name), fontsize=25)
    plt.xlabel("\nGeneration Number\n\n * Population mean accuracy is calculated for each tree only by using data from their train labels.", fontsize=15)
    plt.ylabel("Accuracy\n", fontsize=20)
    plt.legend(loc='lower right', prop={'size': 15})
    plt.show()

In [ ]:
m_X_train, m_X_test, m_y_train, m_y_test = mnist_dataset_read('Datasets/MNIST')
fm_X_train, fm_X_test, fm_y_train, fm_y_test = mnist_dataset_read('Datasets/Fashion_MNIST')
c_X_train, c_y_train, c_X_test, c_y_test = load_data_cifar()

In [ ]:
cgfc = CustomGeneticForestClassifier(N=200, generation_number=10, class_percentage = 0.5)

In [ ]:
m_generation_numbers, m_prediction_tree_accuracies, m_slp_accuracies, m_majority_accuracies, m_population_mean_accuracies = cgfc.model_analysis(m_X_train, m_y_train, m_X_test, m_y_test)

In [ ]:
plot_genetic_analysis(m_generation_numbers, m_prediction_tree_accuracies, m_slp_accuracies, m_majority_accuracies, m_population_mean_accuracies, dataset_name = "MNIST")

In [ ]:
for i in range(len(m_generation_numbers)):
    print("Generation ", m_generation_numbers[i], ":", end=" ") 
    print("\tSLP:{}\tMajority Voting:{}\tPrediction Tree:{}\tPopulation Mean Accuracy:{}".format(m_slp_accuracies[i], m_majority_accuracies[i], m_prediction_tree_accuracies[i], m_population_mean_accuracies[i]))

In [ ]:
cgfc = CustomGeneticForestClassifier(N=200, generation_number=10, class_percentage = 0.5)

In [ ]:
fm_generation_numbers, fm_prediction_tree_accuracies, fm_slp_accuracies, fm_majority_accuracies, fm_population_mean_accuracies = cgfc.model_analysis(fm_X_train, fm_y_train, fm_X_test, fm_y_test)

In [ ]:
plot_genetic_analysis(fm_generation_numbers, fm_prediction_tree_accuracies, fm_slp_accuracies, fm_majority_accuracies, fm_population_mean_accuracies, dataset_name = "Fashion MNIST")

In [ ]:
for i in range(len(fm_generation_numbers)):
    print("Generation ", fm_generation_numbers[i], ":", end=" ") 
    print("\tSLP:{}\tMajority Voting:{}\tPrediction Tree:{}\tPopulation Mean Accuracy:{}".format(fm_slp_accuracies[i], fm_majority_accuracies[i], fm_prediction_tree_accuracies[i], fm_population_mean_accuracies[i]))

In [ ]:
cgfc = CustomGeneticForestClassifier(N=200, generation_number=10, class_percentage = 0.5)

In [ ]:
c_generation_numbers, c_prediction_tree_accuracies, c_slp_accuracies, c_majority_accuracies, c_population_mean_accuracies = cgfc.model_analysis(c_X_train, c_y_train, c_X_test, c_y_test)

In [ ]:
plot_genetic_analysis(c_generation_numbers, c_prediction_tree_accuracies, c_slp_accuracies, c_majority_accuracies, c_population_mean_accuracies, dataset_name = "Cifar-10")

In [ ]:
for i in range(len(c_generation_numbers)):
    print("Generation ", c_generation_numbers[i], ":", end=" ") 
    print("\tSLP:{}\tMajority Voting:{}\tPrediction Tree:{}\tPopulation Mean Accuracy:{}".format(c_slp_accuracies[i], c_majority_accuracies[i], c_prediction_tree_accuracies[i], c_population_mean_accuracies[i]))

In [ ]:
def plot_accuracies(x_values, accuracies, title, x_label):
    plt.figure(figsize=(20, 10))
    for algorithm, accuracies in accuracies.items():
        plt.plot(x_values, accuracies, marker="o", label = algorithm)
    plt.title(title, fontsize=25)
    plt.xlabel("\n{}".format(x_label), fontsize=15)
    plt.ylabel("Accuracy\n", fontsize=20)
    plt.legend(loc='lower right', prop={'size': 15})
    plt.show()

In [ ]:
def get_accuracies(X_train, X_test, y_train, y_test, tree_number, generation_number, class_percentage):
    cgfc = CustomGeneticForestClassifier(N=tree_number, generation_number = generation_number, class_percentage = class_percentage)
    cgfc.fit(X_train, y_train)
    slp_pred=cgfc.slp_predict(X_test)
    prediction_tree_pred=cgfc.prediction_tree_predict(X_test)
    majority_voting_pred=cgfc.majority_voting_predict(X_test)
    
    slp_accuracy = metrics.accuracy_score(y_test, slp_pred)
    prediction_tree_accuracy = metrics.accuracy_score(y_test, prediction_tree_pred)
    majority_voting_accuracy = metrics.accuracy_score(y_test, majority_voting_pred)
    
    return slp_accuracy, majority_voting_accuracy, prediction_tree_accuracy

In [ ]:
def number_of_trees_analysis(N_list, X_train, X_test, y_train, y_test):
    accuracies_of_N_trees_y = {"SLP": [], "Prediction Tree": [], "Majority Voting": [], "Random Forest (scikit-learn)":[]}
    for N in N_list:
        slp_accuracy, majority_voting_accuracy, prediction_tree_accuracy = get_accuracies(X_train, X_test, y_train, y_test, tree_number = N, generation_number = 0, class_percentage = 0.5)
        rf = RandomForestClassifier(n_estimators = N, max_samples = 0.5)
        X_train_rf, X_valid_rf, y_train_rf, y_valid_rf = train_test_split(X_train, y_train, test_size=0.2, random_state=0) 
        rf.fit(X_train_rf, y_train_rf.values.ravel())
        rf_pred = rf.predict(X_test)
        accuracies_of_N_trees_y["Random Forest (scikit-learn)"].append(metrics.accuracy_score(y_test, rf_pred))
        accuracies_of_N_trees_y["SLP"].append(slp_accuracy)
        accuracies_of_N_trees_y["Prediction Tree"].append(prediction_tree_accuracy)
        accuracies_of_N_trees_y["Majority Voting"].append(majority_voting_accuracy)
    return accuracies_of_N_trees_y

In [ ]:
def class_percentage_analysis(class_percentage_list, X_train, X_test, y_train, y_test):
    accuracies_of_class_percentages_y = {"SLP": [], "Prediction Tree": [], "Majority Voting": []}
    for class_percentage in class_percentage_list:
        slp_accuracy, majority_voting_accuracy, prediction_tree_accuracy = get_accuracies(X_train, X_test, y_train, y_test, tree_number = 100, generation_number = 5, class_percentage = class_percentage)
        accuracies_of_class_percentages_y["SLP"].append(slp_accuracy)
        accuracies_of_class_percentages_y["Prediction Tree"].append(prediction_tree_accuracy)
        accuracies_of_class_percentages_y["Majority Voting"].append(majority_voting_accuracy)
    return accuracies_of_class_percentages_y

In [ ]:
N_list = [i*25 for i in range(1,13)]

In [ ]:
m_accuracies_of_N_trees_y = number_of_trees_analysis(N_list, m_X_train, m_X_test, m_y_train, m_y_test)

In [ ]:
plot_title = "Accuracy of Algorithms wrt. Number of Trees (MNIST)\n"
plot_accuracies(N_list, m_accuracies_of_N_trees_y, plot_title, "N Trees")
print(m_accuracies_of_N_trees_y)

In [ ]:
fm_accuracies_of_N_trees_y = number_of_trees_analysis(N_list, fm_X_train, fm_X_test, fm_y_train, fm_y_test)

In [ ]:
plot_title = "Accuracy of Algorithms wrt. Number of Trees (Fashion MNIST)\n"
plot_accuracies(N_list, fm_accuracies_of_N_trees_y, plot_title, "N Trees")
print(fm_accuracies_of_N_trees_y)

In [ ]:
c_accuracies_of_N_trees_y = number_of_trees_analysis(N_list, c_X_train, c_X_test, c_y_train, c_y_test)

In [ ]:
plot_title = "Accuracy of Algorithms wrt. Number of Trees (Cifar-10)\n"
plot_accuracies(N_list, c_accuracies_of_N_trees_y, plot_title, "N Trees")
print(c_accuracies_of_N_trees_y)

In [ ]:
class_percentage_list = [i*0.2 for i in range(1,6)]

In [ ]:
m_accuracies_of_class_percentages_y = class_percentage_analysis(class_percentage_list, m_X_train, m_X_test, m_y_train, m_y_test)

In [ ]:
plot_title = "Accuracy of Algorithms wrt. Class Percentage (MNIST)\n"
plot_accuracies(class_percentage_list, m_accuracies_of_class_percentages_y, plot_title, "Class Percentage")
print(m_accuracies_of_class_percentages_y)

In [ ]:
fm_accuracies_of_class_percentages_y = class_percentage_analysis(class_percentage_list, fm_X_train, fm_X_test, fm_y_train, fm_y_test)

In [ ]:
plot_title = "Accuracy of Algorithms wrt. Class Percentage (Fashion MNIST)\n"
plot_accuracies(class_percentage_list, fm_accuracies_of_class_percentages_y, plot_title, "Class Percentage")
print(fm_accuracies_of_class_percentages_y)

In [ ]:
c_accuracies_of_class_percentages_y = class_percentage_analysis(class_percentage_list, c_X_train, c_X_test, c_y_train, c_y_test)

In [ ]:
plot_title = "Accuracy of Algorithms wrt. Class Percentage (Cifar-10)\n"
plot_accuracies(class_percentage_list, c_accuracies_of_class_percentages_y, plot_title, "Class Percentage")
print(c_accuracies_of_class_percentages_y)